In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

/home/ysx/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
peft_model_id = "/home/ysx/src/AI/llm_demo/outputs/merge-2000"
config = PeftConfig.from_pretrained(peft_model_id)


In [4]:

inference_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True, 
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.53s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
)

In [6]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [7]:
model = inference_model

In [8]:
model = model.eval()

In [9]:
def generate_prompt(instruction, input):
    return (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    )
   

In [10]:
def make_inference(instruction, input):
  question = generate_prompt(instruction, input)
  batch = tokenizer(question, return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = model.generate(**batch, max_new_tokens=200)

  print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In [11]:
i = "解释五行相生和相克之间的关系。"

In [12]:
t = "五行相生和相克是五行理论中的两个基本概念。相生是指五行之间的相互促进和滋养，例如木生火表示木能够促进火的生成。相克则是五行之间的相互制约和克制，例如木克土表示木能够制约土的过度发展。这两种关系在五行理论中是相辅相成的，相生关系保证了五行之间的正向发展，而相克关系则是一种负向调节，防止任何一行的过度发展而破坏整体平衡。"

In [13]:
make_inference(i, t)

/home/ysx/miniconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/ysx/miniconda3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
解释五行相生和相克之间的关系。

### Input:
五行相生和相克是五行理论中的两个基本概念。相生是指五行之间的相互促进和滋养，例如木生火表示木能够促进火的生成。相克则是五行之间的相互制约和克制，例如木克土表示木能够制约土的过度发展。这两种关系在五行理论中是相辅相成的，相生关系保证了五行之间的正向发展，而相克关系则是一种负向调节，防止任何一行的过度发展而破坏整体平衡。

### Response:
五行相生和相克之间的关系是相辅相成的，相生关系促进五行之间的正向相互作用，而相克关系则提供了负向调节，以防止任何一行的过度发展破坏整体的平衡。这两种关系共同维持了五行系统的动态平衡和协调发展。

本次回答已结束。




In [14]:
# merge base model and lora model as a standalone model.
# merged_model = model.merge_and_unload()

In [15]:
tokenizer("111111</s>")

{'input_ids': [1, 29871, 29896, 29896, 29896, 29896, 29896, 29896, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}